In [8]:
from PIL import Image
import numpy as np
import pandas as pd
import tensorflow as tf
from timeit import default_timer as timer
from numba import guvectorize
import cv2

In [29]:
rectSize = 5
inputImagePath = "PostDisaster"
inputImageFile = "channelviewPost1.jpg"

inputImage = cv2.imread(inputImagePath + '/' + inputImageFile)
ipx = inputImage.shape[0]
ipy = inputImage.shape[1]
print(ipx,ipy)

inputImage = cv2.resize(inputImage,(512,512))
ipx = inputImage.shape[0]
ipy = inputImage.shape[1]
print(ipx,ipy)
outputImagePath = "RoadSegments"
outputImageFile = "channelviewPostRoad12.png"
outputImgPixels = np.zeros((ipx,ipy,3), np.uint8)
# print(outputImagePixels)
# outputImagePixels[100,100] = [1,1,1]
opx , opy = ipx, ipy
#ipx, ipy, opx, opy = 100, 100, 100, 100

495 660
512 512


In [34]:
img = cv2.imread("RoadSegments/channelviewPostRoad12.png")
print(img)

[[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 ...

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]]


In [30]:
numeric_columns = []
for i in range(1,76):
    numeric_columns.append("c"+str(i))
#print(numeric_columns)
numeric_features = [ tf.feature_column.numeric_column(key = column)
                    for column in numeric_columns]
#print(numeric_features[0])
trainingSteps = 1000
total = 5000
classifierConfig = tf.estimator.RunConfig(save_checkpoints_secs = None, save_checkpoints_steps = trainingSteps)
hiddenUnits = [100, 150, 100, 50]
classes = 2
modelDir = 'Model3'

classifier = tf.estimator.DNNClassifier(feature_columns = numeric_features,
                                             hidden_units = hiddenUnits,
                                             n_classes = classes,
                                             model_dir = modelDir,
                                             config = classifierConfig)

INFO:tensorflow:Using config: {'_model_dir': 'Model3', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001A924255E48>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [31]:
col = []
l = []
for i in range(1,76):
    col.append("c"+str(i))
print(col)

features = np.zeros((((ipx - ((rectSize//2)*2)) * (ipy - ((rectSize//2)*2))), rectSize*rectSize*3), dtype=np.int)

indexArr = [i for i in range(rectSize//2, opx -(rectSize//2) )]
indexnp = np.array(indexArr)

findexArr = [i for i in range(len(indexArr))]
findexnp = np.array(findexArr)

['c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9', 'c10', 'c11', 'c12', 'c13', 'c14', 'c15', 'c16', 'c17', 'c18', 'c19', 'c20', 'c21', 'c22', 'c23', 'c24', 'c25', 'c26', 'c27', 'c28', 'c29', 'c30', 'c31', 'c32', 'c33', 'c34', 'c35', 'c36', 'c37', 'c38', 'c39', 'c40', 'c41', 'c42', 'c43', 'c44', 'c45', 'c46', 'c47', 'c48', 'c49', 'c50', 'c51', 'c52', 'c53', 'c54', 'c55', 'c56', 'c57', 'c58', 'c59', 'c60', 'c61', 'c62', 'c63', 'c64', 'c65', 'c66', 'c67', 'c68', 'c69', 'c70', 'c71', 'c72', 'c73', 'c74', 'c75']


In [32]:
@guvectorize([("int64[:,:], int64, int64, int64[:,:]")], '(n, m),(),()->(n, m)')
def extractFeatures(ipp, y, f, features):
    #features = np.zeros((ipy - ((rectSize//2)*2), rectSize*rectSize*3), dtype=np.int)
    #features = np.zeros((((1) * (ipy - ((rectSize//2)*2))), rectSize*rectSize*3), dtype=np.int)
    #print(features.shape)
    rowIndex = f * (ipy - ((rectSize//2)*2))
    for x in range(rectSize//2, ipy -(rectSize//2) ):
        rect = (x - (rectSize//2), y - (rectSize//2), x + (rectSize//2) + 1,y + (rectSize//2) + 1 )
        subImage = inputImage[rect[0]:rect[2], rect[1]:rect[3]]
        colIndex = 0       
        for j in range(rectSize):
            for i in range(rectSize):
                features[rowIndex, colIndex] = subImage[i, j][2]
                colIndex += 1
                features[rowIndex, colIndex] = subImage[i, j][1]
                colIndex += 1
                features[rowIndex, colIndex] = subImage[i, j][0]
                colIndex += 1
        rowIndex += 1

                    
def constructSegment(y, f):
    rowIndex =  f * (opy - ((rectSize//2)*2))   
    for x in range(rectSize//2, opy -(rectSize//2)) :
        if(rowIndex >= len(predictions)):
            outputImgPixels[x, y] = [0, 0, 0]
            continue    
        if(predictions[rowIndex]):
            outputImgPixels[x, y] = [255, 255, 255]
            #print("hey")
        else:
            outputImgPixels[x, y] = [0, 0, 0]
        rowIndex += 1
    print(rowIndex)    

    


In [33]:
start = timer()


#parallel
#for i in range(len(indexnp)):
extractFeatures(features, indexnp, findexnp, features)

#serial
x_predict = pd.DataFrame(data = features, columns = col, index = None)
print(x_predict)
predict_input_fn = tf.estimator.inputs.pandas_input_fn(
                    x = x_predict,
                    y = None,
                    batch_size = 32,
                    shuffle = False,
                    num_epochs = 1    
                    )   
predictions = np.array([x['class_ids'][0] for x in classifier.predict(input_fn = predict_input_fn)])
print(predictions)
print(len(predictions))


#row by row
for i in range(len(findexnp)):
    constructSegment(indexnp[i],findexnp[i])
    print("Constructing : %d", i)

cv2.imwrite(outputImagePath + '/' + outputImageFile, outputImgPixels)

#outputImage.save(outputImagePath + '/' + outputImageFile, 'PNG')
#print(l)
#print(len(l))

gpu_time = timer() - start
print(gpu_time)


         c1   c2   c3   c4   c5   c6   c7   c8   c9  c10 ...   c66  c67  c68  \
0        49   61   51   60   72   62   64   76   65   73 ...    76   68   80   
1        60   72   62   64   76   65   73   85   75  101 ...    70   73   85   
2        64   76   65   73   85   75  101  112  104   64 ...    75  107  119   
3        73   85   75  101  112  104   64   75   67   51 ...   110   61   72   
4       101  112  104   64   75   67   51   62   54   58 ...    64   35   46   
5        64   75   67   51   62   54   58   69   61   53 ...    38   47   58   
6        51   62   54   58   69   61   53   64   57   45 ...    50   50   61   
7        58   69   61   53   64   57   45   56   50  134 ...    54   66   77   
8        53   64   57   45   56   50  134  145  140  133 ...    71   68   79   
9        45   56   50  134  145  140  133  143  139   62 ...    75   54   65   
10      134  145  140  133  143  139   62   73   69   45 ...    61   52   63   
11      133  143  139   62   73   69   4

94488
Constructing : %d 185
94996
Constructing : %d 186
95504
Constructing : %d 187
96012
Constructing : %d 188
96520
Constructing : %d 189
97028
Constructing : %d 190
97536
Constructing : %d 191
98044
Constructing : %d 192
98552
Constructing : %d 193
99060
Constructing : %d 194
99568
Constructing : %d 195
100076
Constructing : %d 196
100584
Constructing : %d 197
101092
Constructing : %d 198
101600
Constructing : %d 199
102108
Constructing : %d 200
102616
Constructing : %d 201
103124
Constructing : %d 202
103632
Constructing : %d 203
104140
Constructing : %d 204
104648
Constructing : %d 205
105156
Constructing : %d 206
105664
Constructing : %d 207
106172
Constructing : %d 208
106680
Constructing : %d 209
107188
Constructing : %d 210
107696
Constructing : %d 211
108204
Constructing : %d 212
108712
Constructing : %d 213
109220
Constructing : %d 214
109728
Constructing : %d 215
110236
Constructing : %d 216
110744
Constructing : %d 217
111252
Constructing : %d 218
111760
Constructing : %d 

256032
Constructing : %d 503
256540
Constructing : %d 504
257048
Constructing : %d 505
257556
Constructing : %d 506
258064
Constructing : %d 507
60.79035009999984


In [28]:
import numpy as np
from timeit import default_timer as timer
from numba import vectorize, objmode, njit

M = 30

def myprint(n):
    print(n)
    return n
    
# @vectorize(["float32(float32)"], target='cuda')
# def VectorAdd(a):
#     with objmode(b='float32'):
#         myprint(a)
#     return a

@njit
def foo():
    x = 5
    with objmode(b='float32[:]'):
        myprint(a)
    return a
        
# def main():
#     N = 100
#     A = np.ones(N, dtype=np.float32)
#     B = np.ones(N, dtype=np.float32)
#     C = np.zeros(N, dtype=np.float32)
    
#     start = timer()
#     VectorAdd(A)
#     vectoradd_time = timer() - start
    
#     print("C[:5] = " + str(C[:5]))
#     print("C[-5:] = " + str(C[-5:]))
    
#     print("Vector Add took %f seconds" %vectoradd_time)

# main()  

foo()

AssertionError: Failed at nopython (analyzing bytecode)
SETUP_WITH(arg=14, lineno=20)

In [52]:
import numpy as np
from numba import jit, objmode

def bar():
    print("Hey")
    return np.array([1,2,3])

@jit(nopython = False)
def func():
    x = np.arange(5)
    with objmode(y='intp[:]'):  # annotate return type
        # this region is executed by object-mode.
        y += bar(x)      
    return y
    
func()    

AssertionError: Failed at object (analyzing bytecode)
SETUP_WITH(arg=18, lineno=11)